<a href="http://data.bus.wisc.edu/">
        <img align="right" src="https://emaadmanzoor.com/images/color-UWcrest-print.png" height=80 style="height: 80px; float: right;"/>
</a>

# NLP+CSS 201: Controlling for Text in Causal<br/>Inference with Double Machine Learning

*By [Emaad Manzoor](http://emaadmanzoor.com), [Wisconsin School of Business](https://business.wisc.edu/)*

Slightly updated by Zach Wood-Doughty (March 2025)

**The tutorial slides and video can be found [here](https://nlp-css-201-tutorials.github.io/nlp-css-201-tutorials/).**

## Install required packages

In [1]:
!pip install doubleml==0.9.3
!pip install scikit-learn==1.5.2
!pip install numpy==1.26.4
!pip install pandas
!pip install statsmodels==0.14.4
!pip install econml==0.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.9/342.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 39.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: slicer
    Found existing installation: slicer 0.0.8
    Uninstalling slicer-0.0.8:
      Successfully uninstalled slicer-0.0.8
  Attempting uninstall: shap
    Found existing installation: shap 0.46.0
    Uninstalling shap-0.46.0:
      Successfully uninstalled shap-0.46.0


In [ ]:
import math
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF
from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

from warnings import simplefilter
simplefilter("ignore", category=ConvergenceWarning)

ImportError: cannot import name 'LinearDMLCateEstimator' from 'econml.dml' (/usr/local/lib/python3.11/dist-packages/econml/dml/__init__.py)

In [ ]:
import doubleml
# import econml.dml
# econml.dml.LinearDML
from econml.dml import LinearDML

## Fetch and relabel 20Newsgroups data, construct TF-IDF matrices

Experiment with the TfidfVectorizer parameters.

In [ ]:
newsgroup_data = fetch_20newsgroups(subset="train", remove=("headers", "footers", "quotes"), shuffle=False)

In [ ]:

target_name_to_label = {
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 1,
 'comp.sys.ibm.pc.hardware': 1,
 'comp.sys.mac.hardware': 1,
 'comp.windows.x': 1,
 'misc.forsale': 2,
 'rec.autos': 3,
 'rec.motorcycles': 3,
 'rec.sport.baseball': 4,
 'rec.sport.hockey': 4,
 'sci.crypt': 5,
 'sci.electronics': 6,
 'sci.med': 7,
 'sci.space': 8,
 'alt.atheism': 9,
 'soc.religion.christian': 9,
 'talk.religion.misc': 9,
 'talk.politics.guns': 10,
 'talk.politics.mideast': 10,
 'talk.politics.misc': 10,
}
label_names = ["comp", "sale", "auto", "sport", "crypt", "electronics", "med", "space", "religion", "politics"]
labels = [target_name_to_label[newsgroup_data.target_names[original_label]]
              for original_label in newsgroup_data.target]
labels = np.array(labels)

vectorizer = TfidfVectorizer(min_df=25, max_df=0.01)
tfidf_vectors = vectorizer.fit_transform(newsgroup_data.data).toarray()
print(tfidf_vectors.shape, labels.shape)

documents = []
dropped_document_idx = set([])
vocabset = vectorizer.vocabulary_
for idx, text in enumerate(newsgroup_data.data):
    document = []
    for token in text.split(" "):
        if token not in vocabset:
            continue
        document.append(token)

    if len(document) == 0:
        dropped_document_idx.add(idx)
        continue

    document = " ".join(document)
    if len(document) > 1000:
        document = document[:1000]

    documents.append(document)

tfidf_vectors = np.array([tfidf_vectors[idx, :]
                          for idx in range(tfidf_vectors.shape[0])
                          if idx not in dropped_document_idx])
labels = np.array([labels[idx]
                   for idx in range(len(labels))
                   if idx not in dropped_document_idx])
print(tfidf_vectors.shape, labels.shape, len(documents))

(11314, 3882) (11314,)
(9816, 3882) (9816,) 9816


## Simulate Data for Treatment Effects

For each document $i$, a row of the data consists of:

   * Unobserved confounder $U_i$: Binary, equals 1 if text is from a newsgroup on religion
   * Treatment $Z_i$: depends on $U_i$
   * Outcome $Y_i$: Real-valued, depends on $Z_i$ and $U_i$
   * Document TF-IDF vector $\pmb{X}_i$
   
**The true treatment effect is 0.05.**

You can play around with the manner of simulation (eg. increase the treatment effect size).

In [ ]:
def sigmoid(x):
    return 1. / (1. + math.exp(-x))

simulated_data = []
for idx in tqdm(range(tfidf_vectors.shape[0])):
    # confounder
    U = int(labels[idx]==9) # unobserved confounder = religion topic, influences text

    # treatment
    Z = 2.*U + np.random.normal(loc=0.5, scale=5.0)  # Z depends on U

    # binary outcome
    # yprob = sigmoid(-0.5 + 0.05*Z + 5.0*U)
    # Y = np.random.choice(a=[0, 1],  p=[1.0 - yprob, yprob])
    Y = -0.5 + 0.05*Z + 5.0*U + np.random.normal(0.0, 0.1)

    simulated_data.append([Y, Z, U] + list(tfidf_vectors[idx]))

simulated_data = np.array(simulated_data)

  0%|          | 0/9816 [00:00<?, ?it/s]

## Convert to Pandas Dataframe, Describe Data

In [ ]:
df = pd.DataFrame(simulated_data)
df.columns = ["Y", "Z", "U"] + ["Word" + str(i) for i in range(tfidf_vectors.shape[1])]
df

,Y,Z,U,Word0,Word1,Word2,Word3,Word4,Word5,Word6,...,Word3872,Word3873,Word3874,Word3875,Word3876,Word3877,Word3878,Word3879,Word3880,Word3881
0,-0.550004,-0.175754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,-0.193151,7.361598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.148449,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,-0.585976,-0.033961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.021589,0.0
3,-0.537587,1.309374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,4.364608,-2.363818,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9811,-0.217790,7.760225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9812,-0.493252,-1.178868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9813,-0.658394,-1.663144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9814,-0.358523,2.770806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [ ]:
df[["Y", "Z", "U"]].describe()

,Y,Z,U
count,9816.000000,9816.000000,9816.000000
mean,0.198048,0.702752,0.132641
std,1.750072,5.038643,0.339203
min,-1.490108,-17.178154,0.000000
25%,-0.633092,-2.739312,0.000000
50%,-0.426790,0.663566,0.000000
75%,-0.175845,4.163009,0.000000
max,5.439211,20.845121,1.000000


## Treatment Effect Estimation 1: Regress $Y_i$ on $Z_i$ and $U_i$

Since the estimation includes the unobserved confounder $U_i$, the treatment effect can
be estimated without bias (there are no unobserved confounders).

In [ ]:
%%time
y = simulated_data[:, 0]
X = sm.add_constant(simulated_data[:, 1:3])
model = sm.OLS(endog=y, exog=X)
res = model.fit(method="pinv", maxiter=2000)
print(res.summary(yname="Y", xname=["const", "Z", "U"]))

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.498e+06
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        17:04:10   Log-Likelihood:                 8680.6
No. Observations:                9816   AIC:                        -1.736e+04
Df Residuals:                    9813   BIC:                        -1.733e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4997      0.001   -459.071      0.0

## Treatment Effect Estimation 2: Regress $Y_i$ on $Z_i$ only

Since the estimation does not include the unobserved confounder $U_i$, the estimated treatment effect will
be biased.

In [ ]:
%%time
y = simulated_data[:, 0]
X = sm.add_constant(simulated_data[:, 1:2])
model = sm.OLS(endog=y, exog=X)
res = model.fit(method="pinv", maxiter=100)
print(res.summary(yname="Y", xname=["const", "Z"]))

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     776.9
Date:                Thu, 13 Mar 2025   Prob (F-statistic):          1.32e-164
Time:                        17:04:13   Log-Likelihood:                -19042.
No. Observations:                9816   AIC:                         3.809e+04
Df Residuals:                    9814   BIC:                         3.810e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1288      0.017      7.489      0.0

## Treatment Effect Estimation 3: Regress $Y_i$ on $Z_i$ and $\pmb{X}_i$

Do not run this, it takes too long, and may not even converge!

In [ ]:
# %%time
# y = simulated_data[:, 0]
# X = sm.add_constant(np.hstack((simulated_data[:, 1:2], simulated_data[:, 3:])))
# model = sm.OLS(endog=y, exog=X)
# res = model.fit(method="pinv", maxiter=200)
# res.summary(yname="Y", xname=["const", "Z"])

## Treatment Effect Estimation 4: Regress $Y_i$ on $Z_i$ and Document-Topic Weights

Play around with the number of topics.

In [ ]:
%%time
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

numtopics = 50
# Non-negative Matrix Factorization
nmf = NMF(n_components=numtopics).fit(tfidf_vectors)
tfidf_feature_names = vectorizer.vocabulary_
# print_top_words(nmf, tfidf_feature_names, 10)
nmf_topic_weights = nmf.transform(tfidf_vectors)

CPU times: user 1min 28s, sys: 10.6 s, total: 1min 38s
Wall time: 57.4 s


In [ ]:
%%time
y = simulated_data[:, 0]
X = sm.add_constant(np.hstack((simulated_data[:, 1:2], nmf_topic_weights)))
model = sm.OLS(endog=y, exog=X)
res = model.fit(method="pinv", maxiter=200)
print(res.summary(xname=["const", "Z"] + ["Topic" + str(i+1) for i in range(numtopics)]))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.532
Model:                            OLS   Adj. R-squared:                  0.529
Method:                 Least Squares   F-statistic:                     217.5
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        17:11:15   Log-Likelihood:                -15690.
No. Observations:                9816   AIC:                         3.148e+04
Df Residuals:                    9764   BIC:                         3.186e+04
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0864      0.029      2.978      0.0

## Treatment Effect Estimation 5: Regress $Y_i$ on $Z_i$ and $\pmb{X}_i$ using Double Machine Learning

Play around with the type of ML models used to predict the treatment and outcome.

Note that some models take really long to train (eg. Random Forests).

### Using the [EconML](https://econml.azurewebsites.net/) package

In [ ]:
%%time
Y = simulated_data[:, 0].ravel() # outcome
T = simulated_data[:, 1] # treatment
W = simulated_data[:, 3:] # text

dml_mult = LinearDML(model_y=LassoCV(n_alphas=1, verbose=0, n_jobs=-1),
                                  model_t=LassoCV(n_alphas=1, verbose=0, n_jobs=-1),
                                  linear_first_stages=True)

dml_mult.fit(Y=Y, T=T, W=W, inference="statsmodels")
te_pred_mult = dml_mult.const_marginal_effect().ravel()

lb_mult, ub_mult = dml_mult.const_marginal_effect_interval(alpha=0.05)
lb_mult, ub_mult = lb_mult.ravel(), ub_mult.ravel()

print("Treatment Effect, 95% CI" + "\t\t" + "{:.6f}".format(te_pred_mult[0]) +\
      "({:.6f}".format(lb_mult[0]) + ", " + "{:.6f})".format(ub_mult[0]))

Model LassoCV(cv=2, n_alphas=1, n_jobs=-1, verbose=0) has a non-default cv attribute, which will be ignored
Model LassoCV(cv=2, n_alphas=1, n_jobs=-1, verbose=0) has a non-default cv attribute, which will be ignored


Treatment Effect, 95% CI		0.092546(0.085713, 0.099379)
CPU times: user 26.1 s, sys: 4.58 s, total: 30.7 s
Wall time: 31 s


### Using the [DoubleML](https://docs.doubleml.org/stable/index.html) package

In [ ]:
%%time
data = doubleml.DoubleMLData(data=df, y_col="Y", d_cols="Z",
                             x_cols=["Word" + str(i) for i in range(len(df.columns)-3)])

treatment_predictor = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1)
outcome_predictor = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1)

estimator = doubleml.DoubleMLPLR(data, treatment_predictor, outcome_predictor,
                                 n_folds=2)
estimator.fit()
print(estimator.summary)

       coef   std err          t          P>|t|     2.5 %   97.5 %
Z  0.093565  0.003507  26.677477  8.592985e-157  0.086691  0.10044
CPU times: user 4.53 s, sys: 1.72 s, total: 6.25 s
Wall time: 5.28 s


### From Scratch

Note: Standard errors are different from those recommended in the double machine learning paper.

In [ ]:
%%time
simulated_data_train, simulated_data_inference = train_test_split(simulated_data, test_size=0.5)

Y_train = simulated_data_train[:, 0]
T_train = simulated_data_train[:, 1]
text_train = simulated_data_train[:, 3:]

Y_inference = simulated_data_inference[:, 0]
T_inference = simulated_data_inference[:, 1]
text_inference = simulated_data_inference[:, 3:]

CPU times: user 38.1 ms, sys: 59 ms, total: 97.1 ms
Wall time: 99.7 ms


In [ ]:
%%time
model_y_tr = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1).fit(X=text_train, y=Y_train)
model_t_tr = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1).fit(X=text_train, y=T_train)

CPU times: user 2.08 s, sys: 557 ms, total: 2.64 s
Wall time: 2.41 s


In [ ]:
%%time
model_y_inf = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1).fit(X=text_inference, y=Y_inference)
model_t_inf = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1).fit(X=text_inference, y=T_inference)

CPU times: user 1.94 s, sys: 599 ms, total: 2.54 s
Wall time: 2.68 s


In [ ]:
yres1 = Y_inference - model_y_tr.predict(text_inference)
tres1 = T_inference - model_t_tr.predict(text_inference)
yres2 = Y_train - model_y_inf.predict(text_train)
tres2 = T_train - model_t_inf.predict(text_train)

In [ ]:
theta_1 = np.mean(yres1*tres1)/np.mean(tres1**2)
theta_2 = np.mean(yres2*tres2)/np.mean(tres2**2)
theta = 0.5 * (theta_1 + theta_2)
print("Cross-Fitted Treatment Effect", theta)

Cross-Fitted Treatment Effect 0.09368752092717333


In [ ]:
print("Sample-Split Treatment Effect")
regression_model = sm.OLS(endog=yres1, exog=tres1, hasconst=False)
res = regression_model.fit(method="pinv", maxiter=100)
print(res.summary())

Sample-Split Treatment Effect
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.074
Model:                            OLS   Adj. R-squared (uncentered):              0.074
Method:                 Least Squares   F-statistic:                              392.1
Date:                Thu, 13 Mar 2025   Prob (F-statistic):                    5.16e-84
Time:                        17:13:17   Log-Likelihood:                         -9432.5
No. Observations:                4908   AIC:                                  1.887e+04
Df Residuals:                    4907   BIC:                                  1.887e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------